In [6]:
from ultralytics import YOLO

# download yolo 11n 
model = YOLO('yolo11n.pt')

In [1]:
from pathlib import Path
data_file = Path('Stop-Sign-3/data.yaml')
if data_file.exists():
    print(data_file.read_text())
else:
    print('Stop-Sign-3/data.yaml not found; check the path')

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 9
names: ['Bus', 'Car', 'Crosswalk', 'Crosswalk Signal', 'Pedestrian', 'Speedlimit Signal', 'Stop-signs', 'Trafic Light Signal', 'Truck']

roboflow:
  workspace: weap-cv-team
  project: stop-sign-zn1kw-gqecq
  version: 3
  license: CC BY 4.0
  url: https://universe.roboflow.com/weap-cv-team/stop-sign-zn1kw-gqecq/dataset/3


In [2]:
imgs_dir = Path('Stop-Sign-3/train/images')
labels_dir = Path('Stop-Sign-3/train/labels')

imgs = list(imgs_dir.glob('*')) if imgs_dir.exists() else []
labels = list(labels_dir.glob('*.txt')) if labels_dir.exists() else []

print('images:', len(imgs))
print('labels:', len(labels))

# quick consistency check
img_basenames = {p.stem for p in imgs}
label_basenames = {p.stem for p in labels}

missing_labels = sorted(img_basenames - label_basenames)
extra_labels = sorted(label_basenames - img_basenames)

print('missing label files for images:', len(missing_labels))
print('label files with no image:', len(extra_labels))

if missing_labels:
    print('Examples of images missing labels:', missing_labels[:10])

images: 1041
labels: 1041
missing label files for images: 0
label files with no image: 0


In [3]:
labels_dir = Path('Stop-Sign-3/train/labels')

if labels_dir.exists():
    files = sorted(labels_dir.glob('*.txt'))[:5]  # show up to 5 files
    if not files:
        print('No .txt label files found in', labels_dir)
    for p in files:
        print('---', p.name, '---')
        print(p.read_text())
else:
    print('Stop-Sign-3/train/labels not found; check the path')

--- 00014_00000_00001_png.rf.dfed9eaef10f981cf172a9b215e7bfec.txt ---
6 0.5234375 0.53125 0.69140625 0.72265625
--- 00014_00000_00002_png.rf.852dede93dfe07f167c6e41863573567.txt ---
6 0.4921875 0.53515625 0.71484375 0.75390625
--- 00014_00000_00003_png.rf.5d04c7b7a3dba8a73f5acebbf5cbf63c.txt ---
6 0.49609375 0.5078125 0.69140625 0.71875
--- 00014_00000_00023_png.rf.e8ed8b02c5b911aec2ddd8e4e9c8afa4.txt ---
6 0.515625 0.515625 0.79296875 0.79296875
--- 00014_00000_00024_png.rf.e4d0e93ab7d22e7bbe47da6b28150d51.txt ---
6 0.50390625 0.50390625 0.82421875 0.78515625


In [10]:
# Utilities: modular helpers for validating/exporting the Stage1 model
# Keep this cell small and reusable. Use the functions below when finalizing.
from pathlib import Path
from ultralytics import YOLO
import os

def get_stage1_weights():
    """Return the best or last Stage1 checkpoint path."""
    p_best = Path('runs/detect/finetune_stage1/weights/best.pt')
    p_last = Path('runs/detect/finetune_stage1/weights/last.pt')
    if p_best.exists():
        return str(p_best)
    if p_last.exists():
        return str(p_last)
    raise FileNotFoundError('No Stage1 weights found in runs/detect/finetune_stage1/weights')

def load_model(weights):
    """Load a YOLO model from the given weights path."""
    # Create a YOLO instance directly from the weights file/path.
    return YOLO(str(weights))

def validate(weights, data='Stop-Sign-3/data.yaml', device='cpu'):
    """Run `.val()` for the given weights and return the results object/dict."""
    m = load_model(weights)
    print('Validating', weights)
    # Call .val() without passing a separate `model=` override to avoid config alignment issues
    return m.val(data=data, device=device)

def export_onnx(weights, out_dir='exported', imgsz=640, device='cpu'):
    """Export the given weights to ONNX into `out_dir`."""
    # Load the model directly from the weights file/path and export without extra overrides
    m = load_model(weights)
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    print('Exporting', weights, 'to ONNX (imgsz=', imgsz, ')...')
    # Avoid passing `weights=` or changing `m.overrides` here — call export on the model instance
    m.export(format='onnx', imgsz=imgsz, device=device)
    print('Export complete. Check the working directory for exported files.')

In [7]:
# Training helpers: Stage1 (freeze) and Stage2 (unfreeze)
# These use the same ultralytics API as the utilities above.
from ultralytics import YOLO
from pathlib import Path

def train_stage1(base_weights='yolo11n.pt', data='Stop-Sign-3/data.yaml',
                 epochs=20, imgsz=320, batch=2, device='cpu', name='finetune_stage1'):
    """Run Stage1: freeze backbone, adapt head."""
    print('Stage1: base_weights=', base_weights)
    m = YOLO(str(base_weights))
    m.overrides = getattr(m, 'overrides', {}) or {}
    m.overrides['model'] = str(base_weights)
    m.train(data=data, epochs=epochs, imgsz=imgsz, batch=batch, device=device,
            freeze=[0], augment=True, name=name)
    print('Stage1 completed, outputs in runs/detect/', name)

def train_stage2(start_weights, data='Stop-Sign-3/data.yaml',
                 epochs=30, imgsz=640, batch=2, device='cpu', name='finetune_stage2',
                 auto_augment='randaugment', mixup=0.05, copy_paste=0.2):
    """Run Stage2: unfreeze and fine-tune from start_weights."""
    print('Stage2: starting from', start_weights)
    m = YOLO(str(start_weights))
    m.overrides = getattr(m, 'overrides', {}) or {}
    m.overrides['model'] = str(start_weights)
    m.train(data=data, epochs=epochs, imgsz=imgsz, batch=batch, device=device,
            augment=True, auto_augment=auto_augment, mixup=mixup, copy_paste=copy_paste, name=name)
    print('Stage2 completed, outputs in runs/detect/', name)

# Small helper to continue finetune from Stage1 best if available
def continue_from_stage1(epochs=30, imgsz=640, batch=2, device='cpu'):
    try:
        w = get_stage1_weights()
    except Exception as e:
        print('Stage1 weights not found:', e)
        return
    train_stage2(w, epochs=epochs, imgsz=imgsz, batch=batch, device=device)

# Example usage (uncomment to run):
# train_stage1(base_weights='yolo11n.pt', epochs=10, imgsz=320, batch=2, device='cpu')
# train_stage2('runs/detect/finetune_stage1/weights/best.pt', epochs=30, imgsz=640, batch=2, device='cpu')
# continue_from_stage1(epochs=30)

In [8]:
train_stage1(base_weights='yolo11n.pt', epochs=20, imgsz=320, batch=2, device='cpu')

# RUN_STAGE2 = False  # set True to run Stage2
# if RUN_STAGE2:
#     # Start from Stage1 best checkpoint
#     start = 'runs/detect/finetune_stage1/weights/best.pt'
#     # Adjust epochs/imgsz/batch/device as needed
#     train_stage2(start, epochs=30, imgsz=640, batch=2, device='cpu')
# else:
#     print('Stage2 run skipped. Uncomment and set RUN_STAGE2=True to run.')

Stage1: base_weights= yolo11n.pt
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.5.1 CPU (Apple M2 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Stop-Sign-3/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=[0], half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=finetune_stage1, nbs=64, nms=

/Users/anaconda3/envs/stopSign/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/anaconda3/envs/stopSign/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/anaconda3/envs/stopSign/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/anaconda3/envs/stopSign/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/anaconda3/envs/stopSign/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/anaconda3/envs/stopSign/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/anaconda3/envs/stopSign/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environm

Plotting labels to /Users/HP 1/Documents/WEAP2025/perception/StopSignDetection_YOLO/runs/detect/finetune_stage1/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to /Users/HP 1/Documents/WEAP2025/perception/StopSignDetection_YOLO/runs/detect/finetune_stage1
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/20         0G      1.437      3.435      1.236          2        320: 100% ━━━━━━━━━━━━ 521/521 3.8it/s 2:16<0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 56/56 12.9it/s 4.3s0.2s
                   all        223        864      0.7

In [11]:
# Finalize Stage1 as the final model: validate and export to ONNX
# This cell uses the utilities defined in the previous cell: get_stage1_weights(), validate(), export_onnx()
from pathlib import Path

try:
    stage1 = get_stage1_weights()
except Exception as e:
    print('Stage1 weights not found:', e)
    print('If you want to run Stage2, either provide start weights or re-run Stage1 first.')
else:
    # Validate Stage1 and export the validated weights for deployment
    results = validate(stage1, data='Stop-Sign-3/data.yaml', device='cpu')
    print('Validation results:', results)
    # Export to ./exported (creates directory if needed)
    export_onnx(stage1, out_dir='exported', imgsz=640, device='cpu')
    print('Stage1 validated and exported to ./exported')

# Optional: if you later want to unfreeze and continue fine-tuning, call the function below
# def continue_finetune_from(weights, epochs=30, imgsz=640, batch=2, device='cpu'):
#     m = load_model(weights)
#     m.train(data='Stop-Sign-3/data.yaml', epochs=epochs, imgsz=imgsz, batch=batch, device=device, augment=True, name='finetune_stage2')
# Example usage: continue_finetune_from(stage1, epochs=30)

Validating runs/detect/finetune_stage1/weights/best.pt
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.5.1 CPU (Apple M2 Pro)
YOLO11n summary (fused): 100 layers, 2,583,907 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 24.4±9.0 MB/s, size: 4.0 KB)
val: Scanning /Users/HP 1/Documents/WEAP2025/perception/StopSignDetection_YOLO/Stop-Sign-3/valid/labels.cache... 223 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 223/223 1.3Mit/s 0.0s0s
YOLO11n summary (fused): 100 layers, 2,583,907 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 24.4±9.0 MB/s, size: 4.0 KB)
val: Scanning /Users/HP 1/Documents/WEAP2025/perception/StopSignDetection_YOLO/Stop-Sign-3/valid/labels.cache... 223 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 223/223 1.3Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 1.1it/s 12.6s0.9s
                 Class     Images  